In [1]:
import pandas as pd
import nltk
# nltk.download('all')
import re
import numpy as np
import collections
import sklearn as sk
import string
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

In [2]:
# df = pd.read_csv("../B_Data_Cleaning/synthetic_reviews_all.csv")
df = pd.read_csv("../B_Data_Cleaning/synthetic_reviews_all_trial_1.csv")
df["Review"] = df["Review"].astype("str")
df

Review  \
0      idols hex scifistoryelements airborne underwor...   
1      ablazing ｉｔｓ alanah дрянь ammunition unbreakab...   
2      doubletapping overcrowded critiquer minesweepe...   
3      puffing mild miniboss vybouchnou selected bonl...   
4                  rapid bpoliceb seeen contagion whazoo   
...                                                  ...   
16570  gathering acording damagedealing hquests guide...   
16571  ghameplay houdini viks chews planed ao corpotr...   
16572  oldgen excelentes httpswwwyoutubecomwatchvzfrc...   
16573  preferência fixespolish compli platoon caminan...   
16574  astronomical boards httpsyoutubetowfeueaeeqt p...   

       Recommended or Not Recommended  
0                                True  
1                                True  
2                                True  
3                                True  
4                                True  
...                               ...  
16570                           False  
16571                           False  
16572                           False  
16573                           False  
16574                           False  

[16575 rows x 2 columns]

In [3]:
# Checking the length of all the reviews
(df["Review"]=='').value_counts()

False    16575
Name: Review, dtype: int64

In [4]:
# make a set_column to count the number of words in each review
df["set_column"] = df["Review"].apply(lambda x: set(x.split()))
df

Review  \
0      idols hex scifistoryelements airborne underwor...   
1      ablazing ｉｔｓ alanah дрянь ammunition unbreakab...   
2      doubletapping overcrowded critiquer minesweepe...   
3      puffing mild miniboss vybouchnou selected bonl...   
4                  rapid bpoliceb seeen contagion whazoo   
...                                                  ...   
16570  gathering acording damagedealing hquests guide...   
16571  ghameplay houdini viks chews planed ao corpotr...   
16572  oldgen excelentes httpswwwyoutubecomwatchvzfrc...   
16573  preferência fixespolish compli platoon caminan...   
16574  astronomical boards httpsyoutubetowfeueaeeqt p...   

       Recommended or Not Recommended  \
0                                True   
1                                True   
2                                True   
3                                True   
4                                True   
...                               ...   
16570                           False   
16571                           False   
16572                           False   
16573                           False   
16574                           False   

                                              set_column  
0      {квесты, crashed, versitile, racingsim, sentim...  
1      {systemu, polska, allows, locked, grafics, beh...  
2      {npcplayer, makeshift, goings, tease, zabít, c...  
3      {diverging, teemage, godlike, instadeath, bhad...  
4            {contagion, bpoliceb, rapid, whazoo, seeen}  
...                                                  ...  
16570  {absurde, ويتشر, boba, salvage, frustratingly,...  
16571  {alerting, angly, houdini, interconnect, hscal...  
16572  {steam, skilledlethal, salvage, clipstrailers,...  
16573  {subject, coins, dye, murder, arrives, funded,...  
16574  {plowing, girlfriend, seethrough, offhand, eal...  

[16575 rows x 3 columns]

In [5]:
# split the data into train and test
train, test = sk.model_selection.train_test_split(df, test_size=0.33, random_state=42)

In [6]:
# Dividing dataset into recommended and not recommended
df_recom = train.loc[train["Recommended or Not Recommended"] == True,:]
df_recom = df_recom.reset_index(drop=True)

df_not_recom = train.loc[train["Recommended or Not Recommended"] == False,:]
df_not_recom = df_not_recom.reset_index(drop=True)

In [7]:
# This is to make the set dictionary
# count how many times each word appears in the reviews
bow_recom_temp = collections.Counter([y for x in df_recom.set_column for y in x])
bow_not_recom_temp = collections.Counter([y for x in df_not_recom.set_column for y in x])

bow_recom_set = dict(bow_recom_temp)
bow_not_recom_set = dict(bow_not_recom_temp)

for key in bow_recom_set:
    if key not in bow_not_recom_set:
        bow_not_recom_set[key] = 0
        pass
    else:
        pass

for key in bow_not_recom_set:
    if key not in bow_recom_set:
        bow_recom_set[key] = 0
        pass
    else:
        pass

for key in bow_recom_set:
    bow_recom_set[key] += 1

for key in bow_not_recom_set:
    bow_not_recom_set[key] += 1


In [8]:
# This is to make the list dictionary
# count how many times each word appears in the reviews
bow_recom = collections.Counter([y for x in df_recom.Review for y in x.split()])
bow_not_recom = collections.Counter([y for x in df_not_recom.Review for y in x.split()])

bow_recom_dict = dict(bow_recom)
bow_not_recom_dict = dict(bow_not_recom)

for key in bow_recom_dict:
    if key not in bow_not_recom_dict:
        bow_not_recom_dict[key] = 0
        pass
    else:
        pass

for key in bow_not_recom_dict:
    if key not in bow_recom_dict:
        bow_recom_dict[key] = 0
        pass
    else:
        pass

for key in bow_recom_dict:
    bow_recom_dict[key] += 1

for key in bow_not_recom_dict:
    bow_not_recom_dict[key] += 1
    

In [9]:
bow_recom_sum_vals = sum(bow_recom_dict.values())
bow_not_recom_sum_vals = sum(bow_not_recom_dict.values())

for i in bow_recom_dict:

    bow_recom_dict[i] /= bow_recom_sum_vals

for i in bow_not_recom_dict:

    bow_not_recom_dict[i] /= bow_not_recom_sum_vals

In [10]:
assert len(bow_recom_dict) == len(bow_not_recom_dict)
assert len(bow_recom_set) == len(bow_not_recom_set)

In [11]:
test.drop(columns=["set_column"], inplace=True)

In [12]:
def define_sentiment(element, test_all=False, test_only_tf_abs=False, test_only_idf_abs=False, test_only_prob=False, test_all_abs=False):

    """This function takes a review and returns the label for that review"""

    full_shape = df.shape[0]
    positive_review_probabiliy = len(df_recom)/full_shape
    negative_review_probabiliy = len(df_not_recom)/full_shape
    
    df_choices_positive = [positive_review_probabiliy, bow_recom_set, bow_recom_dict, df_recom]
    df_choices_negative = [negative_review_probabiliy ,bow_not_recom_set, bow_not_recom_dict, df_not_recom]

    def classifier(element, your_class = 'positive'):
        flag = {}
        flag_count = 0
        if your_class == 'positive':
            df_choices = df_choices_positive
        else:
            df_choices = df_choices_negative
        prob_of_class = df_choices[0]/full_shape
        score = 1 * prob_of_class
        score = float(format(score, '.12f'))
        # score = 0.1
        for i in element.split():
            if i not in df_choices[2].keys():
                pass
            else:
                prob_word_given_class = (df_choices[2])[i]
                prob_word_given_class = float(format(prob_word_given_class, '.12f'))
                # Almost the same value, given our spin on this application. 
                # Normally, this term frequency would be calculated differently across the positive and negative documents
                # but we are only looking at the reviews as the documents themselves to determine a word's relevance in the positive
                # or negative corpus. 
                tf = np.log(prob_word_given_class)
                # tf = float(format(tf, '.12f'))
                # tf = abs(np.log(prob_word_given_class))
                # The IDF is the number of reviews / the number of reviews that contain the word in that given corpus
                # idf = abs(np.log(df_choices[3].shape[0]/(df_choices[1])[i]))
                idf = np.log(df_choices[3].shape[0]/(df_choices[1])[i])
                # idf = float(format(idf, '.12f'))
                if test_all:
                    score *= prob_word_given_class*tf*idf
                elif test_only_tf_abs:
                    score *= prob_word_given_class*tf
                    score = abs(score)
                elif test_only_idf_abs:
                    score *= prob_word_given_class*idf
                    score = abs(score)
                elif test_only_prob:
                    score *= prob_word_given_class
                elif test_all_abs:
                    score *= prob_word_given_class*tf*idf
                    score = abs(score)

                if score < 0:
                    flag[flag_count] = (score, your_class)
                    # print(flag_count,i)
                    flag_count += 1
                    # return 'Flag'


        return score
    positive_score = classifier(element, 'positive')
    negative_score = classifier(element, 'negative')
    if positive_score > negative_score:
        return True
    elif positive_score == negative_score:
        # Choosing an arbitrary value, because we assume that a review with one or few words of little substance
        # is implied to be negative, as is usual with netizens. 
        return False
    else:
        return False

In [13]:
test["score_all"] = test.Review.apply(define_sentiment, test_all=True)
test["score_tf"] = test.Review.apply(define_sentiment, test_only_tf_abs=True)
test["score_idf"] = test.Review.apply(define_sentiment, test_only_idf_abs=True)
test["score_freq"] = test.Review.apply(define_sentiment, test_only_prob=True)
test["score_mod_all"] = test.Review.apply(define_sentiment, test_all_abs=True)

In [14]:
print("This is the result with the word 'cyberpunk' included in the corpus:")
print(f"This is our accuracy for (unbalanced) synthetic data with all the formula: {(sum(test['Recommended or Not Recommended'] == test['score_all'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (unbalanced) synthetic data with only class probability, frequency, and absolute value of TF: {(sum(test['Recommended or Not Recommended'] == test['score_tf'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (unbalanced) synthetic data with only class probability, frequency, and absolute value of IDF: {(sum(test['Recommended or Not Recommended'] == test['score_idf'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (unbalanced) synthetic data with only class probability and frequency: {(sum(test['Recommended or Not Recommended'] == test['score_freq'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for (unbalanced) synthetic data with all the formula, but with the absolute value of the score: {(sum(test['Recommended or Not Recommended'] == test['score_mod_all'])/test.shape[0])*100:.2f}%")

This is the result with the word 'cyberpunk' included in the corpus:
This is our accuracy for (unbalanced) synthetic data with all the formula: 46.58%
This is our accuracy for (unbalanced) synthetic data with only class probability, frequency, and absolute value of TF: 86.01%
This is our accuracy for (unbalanced) synthetic data with only class probability, frequency, and absolute value of IDF: 85.34%
This is our accuracy for (unbalanced) synthetic data with only class probability and frequency: 83.13%
This is our accuracy for (unbalanced) synthetic data with all the formula, but with the absolute value of the score: 88.88%


In [15]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_all'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_all']))


              precision    recall  f1-score   support

       False       0.33      0.60      0.43      1805
        True       0.67      0.40      0.50      3665

    accuracy                           0.47      5470
   macro avg       0.50      0.50      0.46      5470
weighted avg       0.56      0.47      0.48      5470



In [16]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_tf'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_tf']))


              precision    recall  f1-score   support

       False       0.71      0.97      0.82      1805
        True       0.98      0.81      0.89      3665

    accuracy                           0.86      5470
   macro avg       0.85      0.89      0.85      5470
weighted avg       0.89      0.86      0.86      5470



In [17]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_idf'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_idf']))


              precision    recall  f1-score   support

       False       0.70      0.96      0.81      1805
        True       0.98      0.80      0.88      3665

    accuracy                           0.85      5470
   macro avg       0.84      0.88      0.85      5470
weighted avg       0.89      0.85      0.86      5470



In [18]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_freq'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_freq']))


              precision    recall  f1-score   support

       False       0.67      0.97      0.79      1805
        True       0.98      0.76      0.86      3665

    accuracy                           0.83      5470
   macro avg       0.82      0.87      0.82      5470
weighted avg       0.88      0.83      0.84      5470

